---
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Cátedra de Investigación Operativa__\
__Autor: Rodrigo Maranzana__, Rmaranzana@frba.utn.edu.ar

---

# Ejemplo de Transporte con Programación Matemática
Se busca transportar por arcos que unen nodos proveedores con nodos clientes, una cantidad determinada de un solo producto. La oferta y demanda de los clientes está balanceada.

In [1]:
import numpy as np
from scipy.optimize import linprog

## Datos del ejemplo
El ejemplo está representado por:
- Una matriz Nodo-Arco.
- Un vector de pesos o costos de los arcos.
- Un vector de oferta y demanda.
- Cotas

In [2]:
# Matriz de adyacencia
Aeq = np.array([[ 1, 1, 0, 0, 0, 0],
                [ 0, 0, 1, 1, 0, 0],
                [ 0, 0, 0, 0, 1, 1],
                [-1, 0,-1, 0,-1, 0],
                [ 0,-1, 0,-1, 0,-1]])

# Vector de costos por arco:
C = np.array([10, 20, 10, 10, 10, 30])

# Vector de oferta y demanda:
beq = np.array([10, 20, 15, -25, -20])

# Cotas:
bounds = tuple([(0, None) for arcs in range(0, C.shape[0])])

# Imprimimos:
print('Matriz Nodo-Arco \n', Aeq,'\n')
print('Costos \n', C,'\n')
print('Oferta/Demanda \n', beq,'\n')
print('Cotas \n', bounds,'\n')

Matriz Nodo-Arco 
 [[ 1  1  0  0  0  0]
 [ 0  0  1  1  0  0]
 [ 0  0  0  0  1  1]
 [-1  0 -1  0 -1  0]
 [ 0 -1  0 -1  0 -1]] 

Costos 
 [10 20 10 10 10 30] 

Oferta/Demanda 
 [ 10  20  15 -25 -20] 

Cotas 
 ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 



## Optimizamos con scipy

In [3]:
# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='revised simplex')

c:\users\54115\appdata\local\programs\python\python37-32\lib\site-packages\scipy\optimize\_linprog_util.py:763: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  warn(redundancy_warning, OptimizeWarning)


## Imprimimos los resultados

In [4]:
print('Cantidad para cada arco:', res.x)
print('Costo mínimo total:', res.fun)

Cantidad para cada arco: [10.  0.  0. 20. 15.  0.]
Costo mínimo total: 450.0


## Resolución con Google OR-Tools

In [3]:
"""
Aplicación de Google OR-Tools para el problema de Transporte en Python.
Ejemplo modificado para el caso de Transporte modelizado como Flujo de Mínimo Costo.
El ejemplo original en inglés se puede encontrar en:

https://developers.google.com/optimization/flow/mincostflow
"""

from ortools.graph import pywrapgraph

# Definir cuatro listas paralelas: nodos inciales, nodos finales, capacidades y costos
# unitarios entre cada par de nodos. Por ejemplo, el arco entre el nodo 0 al nodo 1
# tiene una capacidad Infinita y un costo unitario de 10.

start_nodes = [ 0, 0, 1, 1, 2, 2]
end_nodes   = [ 3, 4, 3, 4, 3, 4]
capacities  = [9999]*6
unit_costs  = [10, 20, 10, 10, 10, 35]


# Definir peso neto de los nodos, oferta y demanda.
supplies = [10, 20, 15, -25, -20]


# Instancear el solver "SimpleMinCostFlow" que resuelve el modelo de Flujo de Mínimo Costo.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Agregar cada arco al problema:
for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i],
                                            capacities[i], unit_costs[i])

# Agregar oferta y demanda:
for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])


# Resolver el problema y obtener la cantidad a enviar por cada arco:
out = min_cost_flow.Solve()

if out == min_cost_flow.OPTIMAL:
    print('Minimum cost:', min_cost_flow.OptimalCost())
    print('')
    print('  Arc    Flow   Cost')
    for i in range(min_cost_flow.NumArcs()):
      cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
      print('%1s -> %1s   %3s    %3s' % (
          min_cost_flow.Tail(i),
          min_cost_flow.Head(i),
          min_cost_flow.Flow(i),
          cost))
else:
    print('Error con el input.')

Minimum cost: 450

  Arc    Flow   Cost
0 -> 3    10    100
0 -> 4     0      0
1 -> 3     0      0
1 -> 4    20    200
2 -> 3    15    150
2 -> 4     0      0
